In [ ]:

def orders_by_vendors():

    query = f"""
    select date_trunc('Month',placed_at) 
        ,global_entity_id
        ,country_code
        ,coalesce(chain_name, vendor_name)
        ,count (distinct order_id) total_orders
    from (
    SELECT
    o.global_entity_id
    ,o.country_code
    ,o.order_id
    ,o.vendor_id
    ,items.name item_name
    ,items.quantity
    ,items.unit_price_eur
    ,v.name vendor_name
    ,v.chain.name chain_name
    ,payment.payment_method
    FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o , UNNEST(items) AS items
    left join  `fulfillment-dwh-production.cl._all_vendors` v on v.vendor_code = o.vendor_id and o.global_entity_id = v.entity_id 
    where partition_date_local between '2024-06-01' and '2024-06-30'
    --and o.global_entity_id = 'MJM_AT'
    and o.vertical_type = 'restaurants'
    --and o.order_id = 'j6es-2423-175e'
    )
    group by 1,2,3
    order by global_entity_id,country_code, total_orders desc
    """

    return query